Öncelikle gerekli olan kütüphanelerimizi ekliyoruz.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn import preprocessing
import missingno as msno
from plotly.offline import init_notebook_mode, iplot, plot
import plotly as py
init_notebook_mode(connected=True)
import plotly.graph_objs as go
from sklearn.model_selection import  GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve, classification_report
from sklearn.metrics import roc_auc_score,roc_curve
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score, f1_score, precision_score
from sklearn.tree import DecisionTreeClassifier
from warnings import filterwarnings
from sklearn.naive_bayes import GaussianNB
import matplotlib.pyplot as plt
from sklearn import ensemble
from sklearn.metrics import confusion_matrix as cm

filterwarnings('ignore')


Şimdi veri çerçevemizi ekleyerek df değişkenine atıyoruz.

In [ ]:
df=pd.read_csv("../input/tabletpc-priceclassification/tablet.csv")

Veri çerçevemizin ilk 5 gözlemini görüntülüyoruz.

In [ ]:
df.head()

Veri çerçevemizin rastgele 5 gözlemini görüntülüyoruz

In [ ]:
df.sample(5)

Veri çerçevemizin son 5 gözlemini görüntülüyoruz

In [ ]:
df.tail()

Veri çerçevemiz hakkında genel bilgilerimizi elde edelim.

In [ ]:
df.info()

Genel bilgilerden gördüğümüz gibi 20 farklı öznitelik 2000 gözlem vardır. Bazı özniteliklerimizin sayısal bazılarının da sayısal olmadığını gözlemliyoruz. 

Şimdi ise özniteliklerimizin isimlerini görüntüleyelim.

In [ ]:
df.columns

Veri çerçevemzin kaç öznitelik , kaç gözlemden oluştuğunu görüntüleyelim.

In [ ]:
df.shape

20 öznitelik 2000 gözlemden oluşmaktadır.

Veri çerçevemizin sayısal özniteliklerinin temel istatistiksel değerlerini görüntüleyelim.

In [ ]:
df.describe()

Öncelikle dikkat ettiğimiz şey ortalamalar ve standart sapmalar arasındaki farklar. İstenilen şey standart sapmanın ortalamara göre düşük olmasıdır. Çünkü standart sapmalarının azlığı değerlerin düzgün olduğunu gösterir. 

OnKameraMP özniteliğinin düzgün dağılmadığı gözlenmektedir.Standart sapması ortalamasından yüksektir.Diğer öznitelikler düzgün bir şekilde dağılım göstermektedir. 


Veri çerçevesinde toplamda kaç adet eksik değer olduğunu gözlemleyelim.

In [ ]:
df.isnull().sum().sum()

Veri çerçevesinde hangi öznitelikte kaç adet eksik değer olduğunu gözlemleyelim.

In [ ]:
df.isnull().sum()

RAM özniteliğinden 12 gözlemde ve OnKameraMP özniteliğinden 5 gözlemin eksik değer olduğunu görüntülüyoruz.

Eksik verilerimizin dağılımı görüntüleyelim.

In [ ]:
msno.matrix(df,figsize=(20, 10));

Göründüğü gibi eksik değerler veri setimizde oldukça azdır. Elle doldurabiliriz. Şimdi eksik değerlerimizin birbirleriyle ilişkilerini ısı haritasıyla görüntüleyelim.

In [ ]:
msno.heatmap(df, figsize= (15,8));

Eksik değerlerin birbirleriyle ilişkisi olmadığını gözlenmektedir. Şimdi eksik değerlerimizi ayrıntılı bir şekilde inceleyelim.

In [ ]:
df[df["RAM"].isnull()]

RAM özniteliğine ait eksik değerlerimizi listelediğimizde hedef öznteliğimiz olan FiyatAraligi özniteliğinin sadece Pahalı olduğu gözlemlerde eksiklik gözlenmiştir.

FiyatAraligi Pahalı olan gözlemlerimizin RAM özniteliğinin temel istatiksel özelliklerini listeleyelim.

In [ ]:
df[df["FiyatAraligi"]=="Pahalı"].describe()["RAM"]

Eksik değerlerimizi ortalama değerleriyle doldurmaya karar veriyoruz. Ver çerçevemizi etkilemeyeceğini düşünüyoruz. Ortalama değerlerini RamOrtalama adlı değişkenimize atıyoruz.

In [ ]:
RamOrtalama=df[df["FiyatAraligi"]=="Pahalı"].mean()["RAM"]

Eksik değerlerimizi ortalama ile dolduruyoruz.

In [ ]:
df['RAM'].fillna(RamOrtalama, inplace = True)

Diğer eksik değer bulunduran özniteliğimiz olan OnKameraMP özniteliğini inceliyoruz.

In [ ]:
df[df["OnKameraMP"].isnull()]

Eksik gözlemler incelendiğinde bu eksik değerlerin hedef değişkenimiz olan FiyatAraligi özniteliğinin Çok Ucuz olduğu noktalarda olduğunu gözlemliyoruz.

FiyatAraligi özniteliğinin Çok Ucuz olduğu noktalardaki temel istatistik değerlerini gözlemliyoruz.

In [ ]:
df[df["FiyatAraligi"]=="Çok Ucuz"].describe()["OnKameraMP"]

Ortalama ile doldurmaya karar veriyoruz. FiyatAraligi çok ucuz olan gözlemlerin OnKameraMP değerlerinin ortalamalarıyla dolduruyoruz.

In [ ]:
OnKameraMPortalama=df[df["FiyatAraligi"]=="Çok Ucuz"].mean()["OnKameraMP"]

In [ ]:
df['OnKameraMP'].fillna(OnKameraMPortalama, inplace = True)

Şimdi eksik verileri doldurma işlemlerinden sonra eksik değer kontrolü yapalım.

In [ ]:
df.isnull().sum().sum()

In [ ]:
df.isnull().sum()

Görüldüğü veri çerçevimizde  gibi eksik değer kalmamıştır

Veri çerçevimizi görüntüleyelim.

In [ ]:
df.head()

Şimdi veri çerçevimizdeki kategorik verilerimizi sayısallaştıralım.

In [ ]:
df.info()

Bluetooth , CiftHat , 4G , 3G ,Dokunmatik , WiFi , FyatAraligi ve Renk gibi gözlemlerin eşsiz değerlerini görüntüleyelim.

In [ ]:
df["Bluetooth"].value_counts()

In [ ]:
df["CiftHat"].value_counts()

In [ ]:
df["4G"].value_counts()

In [ ]:
df["3G"].value_counts()

In [ ]:
df["Dokunmatik"].value_counts()

In [ ]:
df["WiFi"].value_counts()

In [ ]:
df["FiyatAraligi"].value_counts()

Öznitelklerimizi incelediğimizde Bluetooth , CiftHat , 4G , 3G ,Dokunmatik , WiFi özniteliklerinin Var ve Yok değerleri aldığını görüntülüyoruz. Bu değerleri sayısallaştırarak 0-1 olarak değiştiriyoruz.

In [ ]:
df.replace("Yok",0,inplace=True)
df.replace("Var",1,inplace=True)

Veri çerçevemizi görüntüleyelim.

In [ ]:
df.head()

Şimdi de hedef değişkenimiz olan Fiyat Aralığını kategorileştirelim. Değerler arasında üstünlük olduğu için değerlerimizi 0-1-2-3 olarak sayısallaştırıyoruz.

In [ ]:
df["FiyatAraligi"]=df["FiyatAraligi"].replace("Çok Ucuz",0)
df["FiyatAraligi"]=df["FiyatAraligi"].replace("Ucuz",1)
df["FiyatAraligi"]=df["FiyatAraligi"].replace("Normal",2)
df["FiyatAraligi"]=df["FiyatAraligi"].replace("Pahalı",3)

Veri çerçevemizi görüntüleyelim.

In [ ]:
df.head()

Veri çerçevemizin bilgilerini güncelleyelim.

In [ ]:
df.info()

Veri çerçevemizde sadece Renk özniteliğinin object olarak kaldığını görüntülüyoruz.

Renk özniteliğinin eşsiz değerlerini listeleyelim.

In [ ]:
df["Renk"].value_counts()

Renk özniteliğinin hedef değişkenimizdeki etkisini görüntülemek için grafik çizdirelim.

In [ ]:
trace1=go.Bar(
    x=df[df.FiyatAraligi==0].Renk.value_counts().index,
    y=df[df.FiyatAraligi==0].Renk.value_counts().values,
    name="Çok Ucuz",
    marker=dict(color="rgba(200,0,0,0.5)",
               line=dict(color="rgba(0,0,0)",width=2)
               ),
    )

trace2=go.Bar(
    x=df[df.FiyatAraligi==1].Renk.value_counts().index,
    y=df[df.FiyatAraligi==1].Renk.value_counts().values,
    name="Ucuz",
    marker=dict(color="rgba(0,200,0,0.5)",
               line=dict(color="rgba(0,0,0)",width=2)
               ),
    )

trace3=go.Bar(
    x=df[df.FiyatAraligi==2].Renk.value_counts().index,
    y=df[df.FiyatAraligi==2].Renk.value_counts().values,
    name="Normal",
    marker=dict(color="rgba(0,0,200,0.5)",
               line=dict(color="rgba(0,0,0)",width=2)
               ),
    )

trace4=go.Bar(
    x=df[df.FiyatAraligi==3].Renk.value_counts().index,
    y=df[df.FiyatAraligi==3].Renk.value_counts().values,
    name="Pahalı",
    marker=dict(color="rgba(50,20,100,0.5)",
               line=dict(color="rgba(0,0,0)",width=2)
               ),
    )


data=[trace1,trace2,trace3,trace4]
layout= go.Layout(barmode="group")

fig=go.Figure(data=data,layout=layout)
iplot(fig)

Grafiğe bakarak Renk ile FiyatAraligi aralığında ilişki olmadığı gözlenmektedir.

Veri çerçevemizin temel istatistik verilerini tekrar listeyelim.

In [ ]:
df.describe().T

Sayısallaştırılan öznitelikler incelendiğinde 3G ve FiyatAraligi özniteliğinin düzgün dağıldığını diğer sayısallaştırılan özniteliklerin düzgün dağılmadığı görüntülenmektedir.Bunun sebebi ise standart sapmalarının ortalamalarından daha büyük olmasıdır.

Korelasyon katsayılarını görüntüleyelim.

In [ ]:
df.corr()

FiyatAraligi ve RAM arasında güçlü bir pozitif ilişki olduğu gözlenmektedir.ArkaKameraMP - OnKameraMP arasında güçlü bir pozitif ilişki vardır. 3G ve 4G arasında da güçlü bir pozitif ilişki olduğu gözlenmektedir. İlişkileri daha iyi görebilmek için bir ısı haritası çizdirelim.

In [ ]:
f,ax=plt.subplots(figsize=(18,18))
sns.heatmap(df.corr(),annot=True,linewidths=5,fmt='.0%',ax=ax)
plt.show()

Isı haritasında da görüldüğü gibi RAM ve FiyatAralıgı arasında oldukça yüksek ilişki vardır.

Veri çerçevemizi görselleştirerek daha iyi anlamaya çalışalım.

Öncelikle hedef değişkenimizi inceleyelim.

In [ ]:
fig1, ax1 = plt.subplots(1,2,figsize=(8,8))
sns.countplot(df['FiyatAraligi'],ax=ax1[0])
labels = 'ÇOK UCUZ', 'UCUZ', 'NORMAL','PAHALI'
df.FiyatAraligi.value_counts().plot.pie(labels=labels, autopct='%1.1f%%',shadow=True, startangle=90)
plt.show()

Görüldüğü gibi veri çerçevemizde hedef değişkenimiz eşit miktarda dağılmıştır. Düzgün bir dağılım gözlenmektedir.

Aralarında güçlü bir ilişki olan OnKameraMP ve ArkaKameraMP özniteliklerinin grafiğini inceleyelim.

In [ ]:
plt.figure(figsize=(18,5))
sns.lmplot(x="ArkaKameraMP", y="OnKameraMP", data=df)
plt.title('ArkaKameraMP- OnKameraMP')
plt.show()

Görüldüğü gibi bir özniteliğin arttığı noktada diğer özniteliklerin arttığı gçzlenmektedir. Aralarında pozitif bir ilişki gözlenmektedir.

Aralarında pozitif ilişki gözlemlenen bu öznitelikere üçüncü bir öznitelik olarak hedef değişkenimizi ekleyerek etkisini gözlemleyelim.

In [ ]:
plt.figure(figsize=(18,5))
sns.scatterplot(x = "ArkaKameraMP", y = "OnKameraMP", hue="FiyatAraligi", data = df);
plt.title('ArkaKameraMP- OnKameraMP"nın FiyatAraligi Üzerinde Etkisi' )
plt.show()

Özniteliklerimizin arasında pozitif ilişki gözlemlenirken Hedef değişkenimiz olan FiyatAraligi ' nın üzerinde fazla etkileri olmadığı gözlemlenmektedir.

Şimdi de aralarında pozitif ilişki olan CozunurlukGenislik ve CozunurlukYükseklik için bir grafik çizdirelim.

In [ ]:
plt.figure(figsize=(20,5))
sns.scatterplot(x = "CozunurlukGenislik", y = "CozunurlukYükseklik", data = df);
plt.title('CozunurlukGenislik- CozunurlukYükseklik')
plt.show()

Görüldüğü gibi iki değerin az olduğu ve iki değerin fazla olduğu noktalarda yoğunlaşma olduğu içn aralarında pozitif ilişki olduğunu görüntüleyebiliyoruz.

Şimde de hedef değişkenimiz ile diğer özniteliklerimizi karşılaştıralım.

Öncelikle aralarında en güçlü pozitif ilişki bulunan FiyatAraligi ve RAM' in arasıdnaki ilişkiyi görselleştirelim.

In [ ]:
plt.figure(figsize=(18,5))
sns.violinplot(x="FiyatAraligi", y="RAM", data=df)
plt.title('FiyatAraligi- RAM')
plt.show()

Görüldüğü gibi RAM özniteliğinin değerleri arttıkça fiyatında arttığı gözlemlenmektedir.


FiyatAraligi ve BataryaGucu arasındaki ilişkiyi görüntüleyelim.

In [ ]:
plt.figure(figsize=(18,5))
sns.violinplot(x="FiyatAraligi", y="BataryaGucu", data=df)
plt.title('FiyatAraligi- BataryaGucu')
plt.show()

Batarya gücü ile FiyatAraligi arasında ilişki pozitif %20 idi. Bakıldığında çok ucuz olan tabletlerde Batarya gücünün az olduğu pahalı olan tabletlerde ise batarya gücünün daha çok olduğu gözlemler vardır.

FiyatAraligi ve CozunurlukYükseklik arasındaki ilişkiyi görüntüleyelim.

In [ ]:
plt.figure(figsize=(18,5))
sns.violinplot(x = "FiyatAraligi", y = "CozunurlukYükseklik", data = df);
plt.title('FiyatAraligi- CozunurlukYükseklik')
plt.show()

Aralarında güçlü bir ilişki olmadığı gözlenmektedir.

FiyatAraligi ve CozunurlukGenislik arasındaki ilişkiyi görüntüleyelim.

In [ ]:
plt.figure(figsize=(18,5))
sns.violinplot(x = "FiyatAraligi", y = "CozunurlukGenislik", data = df);
plt.title('FiyatAraligi- CozunurlukGenislik')
plt.show()

FiyatAraligi ve CozunurlukGenislik arasında zayıf bir pozitif ilişki gözlenmekteydi. Grafikler incelendiğinde çok ucuz olan tabletlerde CozunurlukGenislik  değerinin az olduğu pahalı olan tabletlerde ise CozunurlukGenislik  değerinin daha çok olduğu gözlemler vardır.

Fiyat Aralıklarına göre çift hat bulundurma sayılarını görselleştirelim.

In [ ]:
trace1=go.Bar(
    x=df[df.FiyatAraligi==0].CiftHat.value_counts().index,
    y=df[df.FiyatAraligi==0].CiftHat.value_counts().values,
    name="Çok Ucuz",
    marker=dict(color="rgba(200,0,0,0.5)",
               line=dict(color="rgba(0,0,0)",width=2)
               ),
    )

trace2=go.Bar(
    x=df[df.FiyatAraligi==1].CiftHat.value_counts().index,
    y=df[df.FiyatAraligi==1].CiftHat.value_counts().values,
    name="Ucuz",
    marker=dict(color="rgba(0,200,0,0.5)",
               line=dict(color="rgba(0,0,0)",width=2)
               ),
    )

trace3=go.Bar(
    x=df[df.FiyatAraligi==2].CiftHat.value_counts().index,
    y=df[df.FiyatAraligi==2].CiftHat.value_counts().values,
    name="Normal",
    marker=dict(color="rgba(0,0,200,0.5)",
               line=dict(color="rgba(0,0,0)",width=2)
               ),
    )

trace4=go.Bar(
    x=df[df.FiyatAraligi==3].CiftHat.value_counts().index,
    y=df[df.FiyatAraligi==3].CiftHat.value_counts().values,
    name="Pahalı",
    marker=dict(color="rgba(50,20,100,0.5)",
               line=dict(color="rgba(0,0,0)",width=2)
               ),
    )


data=[trace1,trace2,trace3,trace4]
layout= go.Layout(barmode="group")

fig=go.Figure(data=data,layout=layout)
iplot(fig)

Çift hatların bulundurduğu tabletlerin daha pahalı olduğu gözlenmektedir. Fakat fiyatta çok az etkili olduğu gözlenmektedir.

Fiyat aralıklarına göre tabletlerin dokunmatik olup olmadığını görselleştirelim.

In [ ]:
trace1=go.Bar(
    x=df[df.FiyatAraligi==0].Dokunmatik.value_counts().index,
    y=df[df.FiyatAraligi==0].Dokunmatik.value_counts().values,
    name="Çok Ucuz",
    marker=dict(color="rgba(200,0,0,0.5)",
               line=dict(color="rgba(0,0,0)",width=2)
               ),
    )

trace2=go.Bar(
    x=df[df.FiyatAraligi==1].Dokunmatik.value_counts().index,
    y=df[df.FiyatAraligi==1].Dokunmatik.value_counts().values,
    name="Ucuz",
    marker=dict(color="rgba(0,200,0,0.5)",
               line=dict(color="rgba(0,0,0)",width=2)
               ),
    )

trace3=go.Bar(
    x=df[df.FiyatAraligi==2].Dokunmatik.value_counts().index,
    y=df[df.FiyatAraligi==2].Dokunmatik.value_counts().values,
    name="Normal",
    marker=dict(color="rgba(0,0,200,0.5)",
               line=dict(color="rgba(0,0,0)",width=2)
               ),
    )

trace4=go.Bar(
    x=df[df.FiyatAraligi==3].Dokunmatik.value_counts().index,
    y=df[df.FiyatAraligi==3].Dokunmatik.value_counts().values,
    name="Pahalı",
    marker=dict(color="rgba(50,20,100,0.5)",
               line=dict(color="rgba(0,0,0)",width=2)
               ),
    )


data=[trace1,trace2,trace3,trace4]
layout= go.Layout(barmode="group")

fig=go.Figure(data=data,layout=layout)
iplot(fig)

Dokunmatik oluşun hedef değişkenimizde çok az etkisi olduğu gözlenmektedir.

Fiyat aralıklarına göre Wifi bulundurma oranlarını görselleştirelim.

In [ ]:
trace1=go.Bar(
    x=df[df.FiyatAraligi==0].WiFi.value_counts().index,
    y=df[df.FiyatAraligi==0].WiFi.value_counts().values,
    name="Çok Ucuz",
    marker=dict(color="rgba(200,0,0,0.5)",
               line=dict(color="rgba(0,0,0)",width=2)
               ),
    )

trace2=go.Bar(
    x=df[df.FiyatAraligi==1].WiFi.value_counts().index,
    y=df[df.FiyatAraligi==1].WiFi.value_counts().values,
    name="Ucuz",
    marker=dict(color="rgba(0,200,0,0.5)",
               line=dict(color="rgba(0,0,0)",width=2)
               ),
    )

trace3=go.Bar(
    x=df[df.FiyatAraligi==2].WiFi.value_counts().index,
    y=df[df.FiyatAraligi==2].WiFi.value_counts().values,
    name="Normal",
    marker=dict(color="rgba(0,0,200,0.5)",
               line=dict(color="rgba(0,0,0)",width=2)
               ),
    )

trace4=go.Bar(
    x=df[df.FiyatAraligi==3].WiFi.value_counts().index,
    y=df[df.FiyatAraligi==3].WiFi.value_counts().values,
    name="Pahalı",
    marker=dict(color="rgba(50,20,100,0.5)",
               line=dict(color="rgba(0,0,0)",width=2)
               ),
    )


data=[trace1,trace2,trace3,trace4]
layout= go.Layout(barmode="group")

fig=go.Figure(data=data,layout=layout)
iplot(fig)

WiFi bulunduran tabletlerin fiyat aralıklarının artttığı gözlenmektedir.

Fiyat aralıklarına göre Bluetooth bulundurma oranlarını görselleştirelim.

In [ ]:
trace1=go.Bar(
    x=df[df.FiyatAraligi==0].Bluetooth.value_counts().index,
    y=df[df.FiyatAraligi==0].Bluetooth.value_counts().values,
    name="Çok Ucuz",
    marker=dict(color="rgba(200,0,0,0.5)",
               line=dict(color="rgba(0,0,0)",width=2)
               ),
    )

trace2=go.Bar(
    x=df[df.FiyatAraligi==1].Bluetooth.value_counts().index,
    y=df[df.FiyatAraligi==1].Bluetooth.value_counts().values,
    name="Ucuz",
    marker=dict(color="rgba(0,200,0,0.5)",
               line=dict(color="rgba(0,0,0)",width=2)
               ),
    )

trace3=go.Bar(
    x=df[df.FiyatAraligi==2].Bluetooth.value_counts().index,
    y=df[df.FiyatAraligi==2].Bluetooth.value_counts().values,
    name="Normal",
    marker=dict(color="rgba(0,0,200,0.5)",
               line=dict(color="rgba(0,0,0)",width=2)
               ),
    )

trace4=go.Bar(
    x=df[df.FiyatAraligi==3].Bluetooth.value_counts().index,
    y=df[df.FiyatAraligi==3].Bluetooth.value_counts().values,
    name="Pahalı",
    marker=dict(color="rgba(50,20,100,0.5)",
               line=dict(color="rgba(0,0,0)",width=2)
               ),
    )


data=[trace1,trace2,trace3,trace4]
layout= go.Layout(barmode="group")

fig=go.Figure(data=data,layout=layout)
iplot(fig)

Bluetooth bulunduran tabletlerin fiyatlarının daha yüksek olduğu gözlenmektedir.

In [ ]:
df = df.rename(columns = {"3G":"G3","4G":"G4"})

Fiyat aralıklarına tabletlerin 3G bulundurmalarının etkisini görselleştirelim.

In [ ]:
trace1=go.Bar(
    x=df[df.FiyatAraligi==0].G3.value_counts().index,
    y=df[df.FiyatAraligi==0].G3.value_counts().values,
    name="Çok Ucuz",
    marker=dict(color="rgba(200,0,0,0.5)",
               line=dict(color="rgba(0,0,0)",width=2)
               ),
    )

trace2=go.Bar(
    x=df[df.FiyatAraligi==1].G3.value_counts().index,
    y=df[df.FiyatAraligi==1].G3.value_counts().values,
    name="Ucuz",
    marker=dict(color="rgba(0,200,0,0.5)",
               line=dict(color="rgba(0,0,0)",width=2)
               ),
    )

trace3=go.Bar(
    x=df[df.FiyatAraligi==2].G3.value_counts().index,
    y=df[df.FiyatAraligi==2].G3.value_counts().values,
    name="Normal",
    marker=dict(color="rgba(0,0,200,0.5)",
               line=dict(color="rgba(0,0,0)",width=2)
               ),
    )

trace4=go.Bar(
    x=df[df.FiyatAraligi==3].G3.value_counts().index,
    y=df[df.FiyatAraligi==3].G3.value_counts().values,
    name="Pahalı",
    marker=dict(color="rgba(50,20,100,0.5)",
               line=dict(color="rgba(0,0,0)",width=2)
               ),
    )


data=[trace1,trace2,trace3,trace4]
layout= go.Layout(barmode="group")

fig=go.Figure(data=data,layout=layout)
iplot(fig)

Tabletlerin 3G bulundurmaları düzgün dağılmamıştı. Bunu önceki işlemlerimizde standart sapmalarına ve ortalamalarına bakarak değerlendirmiştik.  Fiyataralıklarına az derece etki gösterdiğini gözlemlemekteyiz.

In [ ]:
trace1=go.Bar(
    x=df[df.FiyatAraligi==0].G4.value_counts().index,
    y=df[df.FiyatAraligi==0].G4.value_counts().values,
    name="Çok Ucuz",
    marker=dict(color="rgba(200,0,0,0.5)",
               line=dict(color="rgba(0,0,0)",width=2)
               ),
    )

trace2=go.Bar(
    x=df[df.FiyatAraligi==1].G4.value_counts().index,
    y=df[df.FiyatAraligi==1].G4.value_counts().values,
    name="Ucuz",
    marker=dict(color="rgba(0,200,0,0.5)",
               line=dict(color="rgba(0,0,0)",width=2)
               ),
    )

trace3=go.Bar(
    x=df[df.FiyatAraligi==2].G4.value_counts().index,
    y=df[df.FiyatAraligi==2].G4.value_counts().values,
    name="Normal",
    marker=dict(color="rgba(0,0,200,0.5)",
               line=dict(color="rgba(0,0,0)",width=2)
               ),
    )

trace4=go.Bar(
    x=df[df.FiyatAraligi==3].G4.value_counts().index,
    y=df[df.FiyatAraligi==3].G4.value_counts().values,
    name="Pahalı",
    marker=dict(color="rgba(50,20,100,0.5)",
               line=dict(color="rgba(0,0,0)",width=2)
               ),
    )


data=[trace1,trace2,trace3,trace4]
layout= go.Layout(barmode="group")

fig=go.Figure(data=data,layout=layout)
iplot(fig)

4G bulunduran tabletlerin daha pahalı olduğunu görmekteyiz.

In [ ]:
trace1=go.Bar(
    x=df[df.FiyatAraligi==0].CekirdekSayisi.value_counts().index,
    y=df[df.FiyatAraligi==0].CekirdekSayisi.value_counts().values,
    name="Çok Ucuz",
    marker=dict(color="rgba(200,0,0,0.5)",
               line=dict(color="rgba(0,0,0)",width=2)
               ),
    )

trace2=go.Bar(
    x=df[df.FiyatAraligi==1].CekirdekSayisi.value_counts().index,
    y=df[df.FiyatAraligi==1].CekirdekSayisi.value_counts().values,
    name="Ucuz",
    marker=dict(color="rgba(0,200,0,0.5)",
               line=dict(color="rgba(0,0,0)",width=2)
               ),
    )

trace3=go.Bar(
    x=df[df.FiyatAraligi==2].CekirdekSayisi.value_counts().index,
    y=df[df.FiyatAraligi==2].CekirdekSayisi.value_counts().values,
    name="Normal",
    marker=dict(color="rgba(0,0,200,0.5)",
               line=dict(color="rgba(0,0,0)",width=2)
               ),
    )

trace4=go.Bar(
    x=df[df.FiyatAraligi==3].CekirdekSayisi.value_counts().index,
    y=df[df.FiyatAraligi==3].CekirdekSayisi.value_counts().values,
    name="Pahalı",
    marker=dict(color="rgba(50,20,100,0.5)",
               line=dict(color="rgba(0,0,0)",width=2)
               ),
    )


data=[trace1,trace2,trace3,trace4]
layout= go.Layout(barmode="group")

fig=go.Figure(data=data,layout=layout)
iplot(fig)

Grafik incelendğinde çekirdek sayısının fiyat aralığınu büyük miktarda etkilediği düşünülmemektedir.

Görselleştirme süreci tamamlanmıştır. Veri çerçevemizi modeller için hazır hale getirelim.

Hedef değişkenimize etki etmediğini düşündüğümüz Renk değişkenini siliyoruz.

In [ ]:
df=df.drop(["Renk"],axis=1)

Veri çerçevemizin son halini görüntüleyelim.

In [ ]:
df.head()

Bağımlı ve bağımsız özniteliklerimizi bölütleyelim.

In [ ]:
y=df["FiyatAraligi"]

In [ ]:
y

In [ ]:
X=df.drop(['FiyatAraligi'], axis=1)

In [ ]:
X

Veri çerçevemizdeki bağımlı/bağımsız öznitelikleri eğitim ve test olmak üzere bölütleyelim. Bölütleme yüzde 25 oranında yapılacaktır.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test ,y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=1)

Veri çerçevemizi bölütlendirdik. Şimdi Makine öğrenmesi modellerini inşa edelim ve eğitelim.

## Decision Tree Classifier

Desicion Tree Classifier metodunu kullanacağız. Öncelikle parametre olarak criterion değeri otomatik olarak "gini" olarak hesaplama yapacağız. 

In [ ]:
tabletDT=DecisionTreeClassifier()
tabletDT_model=tabletDT.fit(X_train,y_train)

Modelin haberdar olmadığı X_test gözlemlerini tahmin ettirelim ve y_pred'e atayalım.

In [ ]:
y_pred=tabletDT_model.predict(X_test)

Doğruluk skorunu görüntüleyelim.

In [ ]:
accuracy_score(y_test,y_pred)

Modelimiz %84 başarılı skor elde etmiştir. Şimdi sonucu daha iyi anlayabilmek için karmaşıklık matrisini görüntüleyelim.

In [ ]:
cm=confusion_matrix(y_test,y_pred)
cm

Karmaşıklık matrisine bakarak şu sonuçları çıkarıyoruz.
Çok ucuz olması gereken 17 tane değer ucuz değeri almıştır.
Ucuz olması gereken 10 tane tane değer çok ucuz 18 tane değer normal değeri almıştır.
Normal olması gereken 11 tane değer ucuz 10 tane değer pahalı değeri almıştır.
Pahalı olması gereken 11 tane normal değeri almıştır.

Şimdi de parametre olarak criterion='entropy' değişiklik yapıyoruz. Başarı oranının artacağı düşünülmektedir.

In [ ]:
tabletDT=DecisionTreeClassifier(criterion='entropy')
tabletDT_model=tabletDT.fit(X_train,y_train)

In [ ]:
y_pred=tabletDT_model.predict(X_test)

In [ ]:
accuracy_score(y_test,y_pred)

Görüldüğü gibi score değerinde artış olmuştur. Şimdi karmaşıklık matrisine bakarak nasıl bir değişiklik olduğunu görüntüleyelim.

In [ ]:
cm=confusion_matrix(y_test,y_pred)
cm

Karmaşıklık matrisine bakarak şu sonuçları çıkarıyoruz. Çok ucuz olması gereken 15 tane değer ucuz değeri almıştır. Ucuz olması gereken 10 tane tane değer çok ucuz 15 tane değer normal değeri almıştır. Normal olması gereken 9 tane değer ucuz 15 tane değer pahalı değeri almıştır. Pahalı olması gereken 13 tane normal değeri almıştır.

Modeli farklı gözlemlerle kombinleyerek tekrardan skorunu hesaplayalım.

In [ ]:
cross_val_score(tabletDT_model,X,y, cv=10)

In [ ]:
cross_val_score(tabletDT_model,X,y, cv=10).mean()

Görüldüğü gibi farklı kombinlerin ortalaması %85 olarak elde edildi.

Hedef özniteliğin değerlerine göre sınıflandırma raporunu görüntüleyelim

In [ ]:
print(classification_report(y_test, y_pred))

f1-score değerlerine bakıldığında çouk ucuz ve pahalı durumlarında güzel eğitilmiş diğer durumlarda biraz daha iyi olduğu gözlenmektedir. 

Özniteliklerin Karar ağacına göre özniteliklerin önem derecisini görselleştirelim.

In [ ]:
ranking = tabletDT.feature_importances_
features = np.argsort(ranking)[::-1][:10]
columns = X.columns

plt.figure(figsize = (16, 9))
plt.title("Karar Ağacına Göre Özniteliklerin Önem Derecesi", y = 1.03, size = 18)
plt.bar(range(len(features)), ranking[features], color="lime", align="center")
plt.xticks(range(len(features)), columns[features], rotation=80)
plt.show()

RAM özniteliğinin etkisinin diğer özniteliklere göre daha çok olduğu gözlemlenmekedir.

Modelimiz için kritik olan iki parametreyi optimize edip modelimizi tekrar geliştirelim.

In [ ]:
tabletDT_grid={"max_depth": range(1,20),
            "min_samples_split" : range(2,50)}

In [ ]:
dt = DecisionTreeClassifier()
tabletcv = GridSearchCV(dt, tabletDT_grid, cv = 10, n_jobs = -1, verbose = 2)

In [ ]:
tabletcv_model=tabletcv.fit(X_train,y_train)

In [ ]:
print("En iyi parametreler : " + str(tabletcv_model.best_params_))
print("En iyi skor : " + str(tabletcv_model.best_score_))

In [ ]:
tabletdt = DecisionTreeClassifier(max_depth = 8, min_samples_split =5, criterion='entropy')
tablet_dt =tabletdt.fit(X_train, y_train)

In [ ]:
y_pred=tablet_dt.predict(X_test)

In [ ]:
accuracy_score(y_test,y_pred)

In [ ]:
cm=confusion_matrix(y_test,y_pred)
cm

Karmaşıklık matrisine bakarak şu sonuçları çıkarıyoruz. Çok ucuz olması gereken 14 tane değer ucuz değeri almıştır. Ucuz olması gereken 10 tane tane değer çok ucuz 15 tane değer normal değeri almıştır. Normal olması gereken 9 tane değer ucuz 13 tane değer pahalı değeri almıştır. Pahalı olması gereken 12 tane normal değeri almıştır.

## KNN

KNN modelini parametre optimizasyonu yaparak kuralım ve eğitelim

In [ ]:
knn_params = {"n_neighbors": np.arange(1,30)}

Komşular 1 den 30' a kadar denenecek. Ve en iyi komşu değerini göreceğiz.

In [ ]:
knn = KNeighborsClassifier()
knn_cv = GridSearchCV(knn, knn_params, cv = 3)
knn_cv.fit(X_train, y_train)

In [ ]:
print("En iyi skor: " + str(knn_cv.best_score_))
print("En iyi parametreler: " + str(knn_cv.best_params_))

En iyi komşu değerinin 19 olduğu gözlenmektedir. 

In [ ]:
knn = KNeighborsClassifier(19)
knn_tuned = knn.fit(X_train, y_train)

In [ ]:
y_pred = knn_tuned.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

Doğruluk skorunun % 92 olduğu gözlenmektedir. Bu sonucu daha iyi değerlendirebilmek için Karmaşıklık Matrisi çizdiriyoruz.

In [ ]:
cm=confusion_matrix(y_test,y_pred)
cm

Karmaşıklık matrisine bakarak şu sonuçları çıkarıyoruz. Çok ucuz olması gereken 6 tane değer ucuz değeri almıştır. Ucuz olması gereken 2 tane tane değer çok ucuz 7 tane değer normal değeri almıştır. Normal olması gereken 6 tane değer ucuz 7 tane değer pahalı değeri almıştır. Pahalı olması gereken 9 tane normal değeri almıştır. Bu sonuca bakarak Desicion Tree modeline göre daha başarılı bir sonuç elde edilmiştir.

Komşuluk değerlerine göre doğruluk skorunun değişimini görselleştirelim.

In [ ]:
score_list = []

for each in range(1,16):
    knn2 = KNeighborsClassifier(n_neighbors = each)
    knn2.fit(X_train,y_train)
    score_list.append(knn2.score(X_test, y_test))
    
plt.plot(range(1,16),score_list)
plt.xlabel("k değerleri")
plt.ylabel("doğruluk skoru")
plt.show()

11 değerinde daha yüksek bir sonuç elde edilmiştir. k değerini 11 yaparak modelimizi tekrar inşaa edelim.

In [ ]:
knn = KNeighborsClassifier(11)
knn_tuned = knn.fit(X_train, y_train)

In [ ]:
y_pred = knn_tuned.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
cm=confusion_matrix(y_test,y_pred)
cm

Karmaşıklık matrisine bakarak şu sonuçları çıkarıyoruz. Çok ucuz olması gereken 6 tane değer ucuz değeri almıştır. Ucuz olması gereken 3 tane tane değer çok ucuz 6 tane değer normal değeri almıştır. Normal olması gereken 4 tane değer ucuz 5 tane değer pahalı değeri almıştır. Pahalı olması gereken 4 tane normal değeri almıştır. Önceki modellerimize göre daha iyi bir sonuç elde ettik.

Farklı kombinlerle sonuç oranlarımızı görelim ve ortalamalarınu görüntüleyelim.

In [ ]:
cross_val_score(knn_tuned, X_test, y_test, cv = 10)

In [ ]:
cross_val_score(knn_tuned, X_test, y_test, cv = 10).mean()

Hedef özniteliğin değerlerine göre sınıflandırma raporunu görüntüleyelim.

In [ ]:
print(classification_report(y_test, y_pred))

f1-score değerlerine baktığımızda modelimiz bütün hedef öznitelikler için ortalama %94 oranında  güzel eğitilmiş . 

## Naive Bayes Algoritması

GaussianNB modeli yaratalım ve modelin eğitimini "tüm gözlemler üzerinde" gerçekleştirelim.

In [ ]:
nb = GaussianNB()

In [ ]:
nb_model = nb.fit(X_train, y_train)

Veri çerçevesindeki tüm gözlemleri tahmin edip y_pred değişkenine atayalım.

In [ ]:
y_pred = nb_model.predict(X_test)

Doğruluk oranımızı görüntüleyelim.

In [ ]:
accuracy_score(y_test, y_pred)

Yüzde %81 oranında doğruluk oranı vardır.

Sonucumuzu daha iyi görebilmek için karmaşıklık matrisini görüntüleyelim.

In [ ]:
karmasiklik_matrisi = confusion_matrix(y_test, y_pred)
karmasiklik_matrisi

Karmaşıklık matrisine bakarak şu sonuçları çıkarıyoruz. Çok ucuz olması gereken 15 tane değer ucuz değeri almıştır. Ucuz olması gereken 9 tane tane değer çok ucuz 28 tane değer normal değeri almıştır. Normal olması gereken 17 tane değer ucuz 14 tane değer pahalı değeri almıştır. Pahalı olması gereken 11 tane normal değeri ve 1 tane ucuz değeri almıştır. 

Şimdi farklı kombinlerdeki sonuçlarımızı görüntüleyelim.

In [ ]:
cross_val_score(nb,X_test,y_test,cv=10)

In [ ]:
cross_val_score(nb,X_test,y_test,cv=10).mean()